In [1]:
import openpyxl
import glob
from pathlib import Path

In [2]:
import cv2 as cv
import numpy as np
from sklearn.mixture import GaussianMixture
import matplotlib.pyplot as plt
from scipy.signal import find_peaks
import seaborn as sns
from scipy import stats
import os
import re
import pickle
import imutils
from scipy.signal import find_peaks, savgol_filter
from pathlib import Path
import glob
from pathlib import Path
import pandas as pd
from tqdm import tqdm
from utils import init_condition, select_roi_interactive, load_significant_frames
from utils import noise_detection,  cfar_fast
from utils import find_periods

In [3]:
trials = []
# Define variable to load the dataframe
dataframe = openpyxl.load_workbook("F:\\swim_motion_data\\selected_sets.xlsx")

# Define variable to read sheet
dataframe1 = dataframe.active

# Iterate the loop to read the cell values
for row in range(0, dataframe1.max_row):
    for col in dataframe1.iter_cols(1, 1):
        trials.append(col[row].value)

In [4]:
for trial in trials:
    prefix = trial[5:].split("-T")[0]
    trial_no = trial.split("T")[-1].split("-")[0]
    date = trial.split("_")[1]
    Fx = trial.split("_")[2]
    
    mp4 = glob.glob(f"F:\\Temp_opto\\{date}\\**\\TOPCAMERA\\{Fx}\\**\\{prefix}_Trial{trial_no}.mp4")[0]
    
    save_dir = f"F:\\swim_motion_data\\lcr_kai\\{trial[5:-5]}\\TOPCAM"
    Path(save_dir).mkdir(parents = True, exist_ok = True)

    if Path(f"{save_dir}/cord.txt").exists():
       with open(f"{save_dir}/cord.txt", "r") as f:
           x_cord = int(f.readline().split()[-1])
           y_cord = int(f.readline().split()[-1])
    else:
        import winsound
        frequency = 2500  # Set Frequency To 2500 Hertz
        duration = 500  # Set Duration To 1000 ms == 1 second
        winsound.Beep(frequency, duration)
        x_cord, y_cord = select_roi_interactive(mp4)
        with open(f"{save_dir}/cord.txt", "w") as f:
            f.write("x\t" + str(x_cord))
            f.write("\ny\t" + str(y_cord))

    capture, num_frames, hsv = init_condition(mp4, x_cord, y_cord)

    if Path(f"{save_dir}/y_shift.npy").exists():
        vertical_motion = np.load(f"{save_dir}/y_shift.npy")
    else:
        vertical_motion = []
    
        capture.set(cv.CAP_PROP_POS_FRAMES, 0)
        ret, prev_frame = capture.read()
        prev_frame = prev_frame[y_cord:y_cord+50, x_cord:x_cord+50]
        prev_gray = cv.cvtColor(prev_frame, cv.COLOR_BGR2GRAY)
        
        for frame in tqdm(range(1, num_frames), desc=f"Optic Flow for {trial[5:-5]}"):
            ret, curr_frame = capture.read()
            if not ret:
                break
                
            curr_frame = curr_frame[y_cord:y_cord+50, x_cord:x_cord+50]
            curr_gray = cv.cvtColor(curr_frame, cv.COLOR_BGR2GRAY)
            
            # Calculate dense optical flow
            flow = cv.calcOpticalFlowFarneback(
                prev_gray, curr_gray, 
                None, 0.5, 3, 15, 3, 5, 1.2, 0
            )
            
            # Focus on vertical motion (y-axis component)
            vertical_flow = flow[..., 1]
            avg_vertical = np.mean(vertical_flow)
            vertical_motion.append(avg_vertical)
            
            prev_gray = curr_gray
        vertical_motion = -np.array(vertical_motion) ###### make positive to upward motion 
        np.save(f"{save_dir}/y_shift.npy",vertical_motion)

Optic Flow for 2024-08-29_F4_01-T11: 100%|█████████████████████████████████████| 72000/72000 [00:58<00:00, 1223.78it/s]
